# Renaming as per csv

In [1]:
import os
import pandas as pd
from pathlib import Path
import shutil

def rename_files_from_csv(file_path, csv_path, dry_run=True):
    """
    Rename files based on a CSV mapping.
    
    Parameters:
    file_path (str): Path to the directory containing files to rename
    csv_path (str): Path to the CSV file with old_name, new_name columns
    dry_run (bool): If True, only show what would be renamed without actually renaming
    
    Returns:
    dict: Results of the renaming operation
    """
    
    # Convert paths to Path objects
    file_dir = Path(file_path)
    csv_file = Path(csv_path)
    
    # Validate inputs
    if not file_dir.exists():
        return {"error": f"Directory not found: {file_path}"}
    
    if not csv_file.exists():
        return {"error": f"CSV file not found: {csv_path}"}
    
    try:
        # Read CSV file
        # Assuming CSV has two columns: original filename and new filename
        df = pd.read_csv(csv_file, header=None, names=['original', 'new'])
        
        # Remove any whitespace from column values
        df['original'] = df['original'].str.strip()
        df['new'] = df['new'].str.strip()
        
        results = {
            "successful": [],
            "failed": [],
            "not_found": [],
            "total_processed": 0
        }
        
        print(f"{'DRY RUN - ' if dry_run else ''}File Renaming Operation")
        print("=" * 50)
        
        for index, row in df.iterrows():
            original_name = row['original']
            new_name = row['new']
            
            original_path = file_dir / original_name
            new_path = file_dir / new_name
            
            results["total_processed"] += 1
            
            # Check if original file exists
            if not original_path.exists():
                print(f"❌ File not found: {original_name}")
                results["not_found"].append(original_name)
                continue
            
            # Check if new filename already exists
            if new_path.exists() and original_path != new_path:
                print(f"⚠️  Target file already exists: {new_name}")
                results["failed"].append({
                    "original": original_name,
                    "new": new_name,
                    "reason": "Target file already exists"
                })
                continue
            
            try:
                if dry_run:
                    print(f"🔍 Would rename: {original_name} → {new_name}")
                else:
                    # Perform the actual rename
                    original_path.rename(new_path)
                    print(f"✅ Renamed: {original_name} → {new_name}")
                
                results["successful"].append({
                    "original": original_name,
                    "new": new_name
                })
                
            except Exception as e:
                print(f"❌ Failed to rename {original_name}: {str(e)}")
                results["failed"].append({
                    "original": original_name,
                    "new": new_name,
                    "reason": str(e)
                })
        
        # Print summary
        print("\n" + "=" * 50)
        print("SUMMARY:")
        print(f"Total files processed: {results['total_processed']}")
        print(f"Successful: {len(results['successful'])}")
        print(f"Failed: {len(results['failed'])}")
        print(f"Not found: {len(results['not_found'])}")
        
        if dry_run:
            print("\n💡 This was a dry run. Set dry_run=False to actually rename files.")
        
        return results
        
    except Exception as e:
        return {"error": f"Error reading CSV file: {str(e)}"}

# Example usage:
def example_usage():
    """
    Example of how to use the file renaming function
    """
    
    # Example 1: Dry run (recommended first)
    print("Example 1: Dry run")
    file_directory = "/path/to/your/files"  # Replace with your file directory
    csv_mapping = "/path/to/your/mapping.csv"  # Replace with your CSV file path
    
    # First, do a dry run to see what would happen
    results = rename_files_from_csv(file_directory, csv_mapping, dry_run=True)
    
    if "error" in results:
        print(f"Error: {results['error']}")
    else:
        print("Dry run completed successfully!")
    
    print("\n" + "-" * 50 + "\n")
    
    # Example 2: Actual renaming (uncomment when ready)
    # print("Example 2: Actual renaming")
    # results = rename_files_from_csv(file_directory, csv_mapping, dry_run=False)
    
    # if "error" in results:
    #     print(f"Error: {results['error']}")
    # else:
    #     print("File renaming completed!")

# Helper function to create a backup
def create_backup(file_path, backup_path=None):
    """
    Create a backup of the directory before renaming files
    
    Parameters:
    file_path (str): Path to the directory to backup
    backup_path (str): Path where backup should be created (optional)
    """
    
    file_dir = Path(file_path)
    
    if backup_path is None:
        backup_path = file_dir.parent / f"{file_dir.name}_backup"
    else:
        backup_path = Path(backup_path)
    
    try:
        shutil.copytree(file_dir, backup_path)
        print(f"✅ Backup created at: {backup_path}")
        return str(backup_path)
    except Exception as e:
        print(f"❌ Failed to create backup: {str(e)}")
        return None

# Quick start function
def quick_rename(file_path, csv_path, create_backup_first=True):
    """
    Quick start function that handles the complete workflow
    """
    
    print("🚀 Quick File Rename Utility")
    print("=" * 40)
    
    # Create backup if requested
    if create_backup_first:
        print("Creating backup...")
        backup_location = create_backup(file_path)
        if backup_location is None:
            print("❌ Backup failed. Aborting operation for safety.")
            return
    
    # First do a dry run
    print("\n📋 Performing dry run...")
    dry_results = rename_files_from_csv(file_path, csv_path, dry_run=True)
    
    if "error" in dry_results:
        print(f"❌ Error during dry run: {dry_results['error']}")
        return
    
    # Ask for confirmation
    print(f"\n🤔 Ready to rename {len(dry_results['successful'])} files?")
    print("   Type 'yes' to proceed, anything else to cancel:")
    
    # Note: In Jupyter, you might want to replace this with a direct call
    # confirmation = input().lower().strip()
    confirmation = "yes"  # Remove this line and uncomment above for interactive use
    
    if confirmation == "yes":
        print("\n🔄 Performing actual file renaming...")
        final_results = rename_files_from_csv(file_path, csv_path, dry_run=False)
        
        if "error" in final_results:
            print(f"❌ Error during renaming: {final_results['error']}")
        else:
            print("🎉 File renaming completed successfully!")
    else:
        print("❌ Operation cancelled by user.")

In [ ]:
# Uncomment and modify these lines to use the script:
file_directory = r"D:\Sakib\dataset\wavs"
csv_mapping_file = r"D:\Sakib\renaming.csv"
quick_rename(file_directory, csv_mapping_file, create_backup_first=False)

🚀 Quick File Rename Utility

📋 Performing dry run...
DRY RUN - File Renaming Operation
🔍 Would rename: A-1.wav → 1.wav
🔍 Would rename: A-2.wav → 2.wav
🔍 Would rename: A-3.wav → 3.wav
🔍 Would rename: A-4.wav → 4.wav
🔍 Would rename: A-5.wav → 5.wav
🔍 Would rename: A-6.wav → 6.wav
🔍 Would rename: A-7.wav → 7.wav
🔍 Would rename: A-8.wav → 8.wav
🔍 Would rename: A-9.wav → 9.wav
🔍 Would rename: A-10.wav → 10.wav
🔍 Would rename: A-11.wav → 11.wav
🔍 Would rename: A-12.wav → 12.wav
🔍 Would rename: A-13.wav → 13.wav
🔍 Would rename: A-14.wav → 14.wav
🔍 Would rename: A-15.wav → 15.wav
🔍 Would rename: A-16.wav → 16.wav
🔍 Would rename: A-17.wav → 17.wav
🔍 Would rename: A-18.wav → 18.wav
🔍 Would rename: A-19.wav → 19.wav
🔍 Would rename: A-20.wav → 20.wav
🔍 Would rename: A-21.wav → 21.wav
🔍 Would rename: A-22.wav → 22.wav
🔍 Would rename: A-23.wav → 23.wav
🔍 Would rename: A-24.wav → 24.wav
🔍 Would rename: A-25.wav → 25.wav
🔍 Would rename: A-26.wav → 26.wav
🔍 Would rename: A-27.wav → 27.wav
🔍 Would renam